<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-fundamentals/unit01-pytorch-tensors/01_linear_algebra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Code Efficiency with Linear Algebra

**Reference**

[Code Efficiency with Linear Algebra](https://lightning.ai/pages/courses/deep-learning-fundamentals/2-0-unit-2-overview/2-4-improving-code-efficiency-with-linear-algebra-parts-1-4/)

[Deep Learning Fundamentals: Code Materials and Exercises](https://github.com/Lightning-AI/dl-fundamentals)

##Setup

In [1]:
import random
import pandas as pd
import numpy as np
import torch

%matplotlib inline
import matplotlib.pyplot as plt

##Part 1: From Loops to Dot Products

In [ ]:
b = 0.0
x = [1.2, 2.2]
w = [3.3, 4.3]

output = b
for x_i, w_i in zip(x, w):
  output += x_i * w_i
print(output)

13.42


In [ ]:
b = torch.tensor([0.0])
x = torch.tensor([1.2, 2.2])
w = torch.tensor([3.3, 4.3])

output = x.dot(w) + b
print(output)

tensor([13.4200])


###Benchmark

In [ ]:
def plain_python(x, w, b):
  output = b
  for x_i, w_i in zip(x, w):
    output += x_i * w_i
  return output

In [ ]:
random.seed(123)

b = 0.0
x = [random.random() for _ in range(1000)]
w = [random.random() for _ in range(1000)]

In [ ]:
%%timeit

plain_python(x, w, b)

120 µs ± 36.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
def pytorch_dot(x, w, b):
  return x.dot(w) + b

In [ ]:
t_b = torch.tensor(b)
t_x = torch.tensor(x)
t_w = torch.tensor(w)

In [ ]:
%%timeit

pytorch_dot(t_x, t_w, t_b)

6.82 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
# excercise
a = torch.tensor([1.2, 5.1, -4.6])
b = torch.tensor([-2.1, 3.1, 5.5])

a.dot(b)

tensor(-12.0100)

##Part 2:Matrix-Vector Multiplication

In [ ]:
b = 0.0
X = [
  [1.2, 2.2],
  [4.4, 5.5]
]
w = [3.3, 4.3]

outputs = []
for x in X:
  output = b
  for x_j, w_j in zip(x, w):
    output += x_j * w_j
  outputs.append(output)
outputs 

[13.42, 38.17]

In [ ]:
b = torch.tensor([0.0])
X = torch.tensor([
  [1.2, 2.2],
  [4.4, 5.5]
])
w = torch.tensor([3.3, 4.3])

X.matmul(w) + b

tensor([13.4200, 38.1700])

###Benchmark

In [ ]:
random.seed(123)

b = 0.0
# 1000 coloumns and  500 rows
X = [[random.random() for _ in range(1000)] for i in range(500)]
w = [random.random() for _ in range(1000)]

def plain_python(X, w, b):
  outputs = []
  for x in X:
    output = b
    for x_j, w_j in zip(x, w):
      output += x_j * w_j
    outputs.append(output)
  return outputs 

In [ ]:
%timeit plain_python(X, w, b)

33 ms ± 584 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
t_b = torch.tensor(b)
t_x = torch.tensor(X)
t_w = torch.tensor(w)

def pytorch_implementation(X, w, b):
  return X.matmul(w) + b

In [ ]:
%timeit pytorch_implementation(t_x, t_w, t_b)

50.1 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


##Part 3:Matrix-Matrix Multiplication

In [2]:
X = torch.rand(100, 10)
W = torch.rand(50, 10)

R = torch.matmul(X, W.T)
R.shape

torch.Size([100, 50])

In [7]:
# excercise
A = torch.tensor([
  [1, 2],
  [2, 4]
])
B = torch.tensor([
  [5, 6],
  [7, 8]
])

C = torch.matmul(A, B.T)
print(C.shape)
C

torch.Size([2, 2])


tensor([[17, 23],
        [34, 46]])

##Part 4: Broadcasting

In [8]:
a = torch.tensor([1.1, 2.1, 3.1, 4.1])
b = torch.tensor([5.4, 5.5, 5.6, 5.7])

a + b

tensor([6.5000, 7.6000, 8.7000, 9.8000])

In [9]:
a = torch.tensor([1.1, 2.1, 3.1, 4.1])
b = torch.tensor([5.4]) # it is broadcasted

a + b

tensor([6.5000, 7.5000, 8.5000, 9.5000])

In [10]:
A = torch.tensor([
  [1.1, 2.1, 3.1, 4.1],
  [1.2, 2.2, 3.2, 4.2]
])
b = torch.tensor([
  [5.4, 5.5, 5.6, 5.7],
  [5.4, 5.5, 5.6, 5.7]
])

A + b

tensor([[6.5000, 7.6000, 8.7000, 9.8000],
        [6.6000, 7.7000, 8.8000, 9.9000]])

In [12]:
A = torch.tensor([
  [1.1, 2.1, 3.1, 4.1],
  [1.2, 2.2, 3.2, 4.2]
])
b = torch.tensor([
  [5.4, 5.5, 5.6, 5.7],
  # it is broadcasted
])

A + b

tensor([[6.5000, 7.6000, 8.7000, 9.8000],
        [6.6000, 7.7000, 8.8000, 9.9000]])

In [14]:
# excercise
a = torch.tensor([
  [1., 2.],
  [3., 4.]
])

a + 1.0

tensor([[2., 3.],
        [4., 5.]])

In [15]:
a * 2

tensor([[2., 4.],
        [6., 8.]])

In [16]:
a + torch.tensor([1., 2.])

tensor([[2., 4.],
        [4., 6.]])

In [17]:
a + torch.tensor([[5., 6.], [7., 8.]])

tensor([[ 6.,  8.],
        [10., 12.]])

In [18]:
a + torch.tensor([[5., 6., 7., 8.]])

RuntimeError: ignored